In [1]:
# 导入django环境
import django
import sys, os
pwd = !pwd  # 项目位置
path = pwd[0]
settings = '%s.settings' % path.split('/')[-1]
sys.path.append(path)
os.chdir(path)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", settings)

'IndexConstDog.settings'

In [3]:
from __future__ import division
from subprocess import call
import pandas as pd

In [14]:
from indexconst.models import Index
from indexconst.models import IndexConst

In [17]:
def flashIndexConst(index):
    """
    更新指数的成分数据
    """
    #  获取链接信息
    filename = 'weight.xls'
    url = index.loadUrl

    # 下载文件
    call(['rm', filename])
    result = call(['wget', url, '-O', filename])
    if result != 0:
        raise Exception(u'下载文件失败')

    # 读取文件并转化格式
    df = pd.io.excel.read_excel(filename)
    df = df[['Date', 'Constituent Code', 'Weight(%)']]
    df.columns = ['date', 'code', 'weight']
    df['code'] = df.code.apply(lambda x: str(x).zfill(6))
    df['weight']  = df['weight'] / 100

    # 保存到参数表
    for i, row in df.iterrows():
        date = row['date']
        code = row['code']
        weight = row['weight']
        IndexConst(index=index, date=date, code=code, weight=weight).save()

In [19]:
for index in  Index.objects.all():
    print u'%s:开始更新' % index.name
    try:
        flashIndexConst(index)
        print u'%s:更新完成' % index.name
    except:
         print u'%s:出错...' % index.name    

沪深300:开始更新
沪深300:更新完成
中证500:开始更新
中证500:更新完成
